In [2]:
import os 
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [3]:
lt_logs_df = pd.read_csv('./logs/long_term_forecast.csv', index_col = 0)
from types import SimpleNamespace
ett_args = SimpleNamespace(**lt_logs_df.loc[2].to_dict())
usc_args = SimpleNamespace(**lt_logs_df.loc[26].to_dict())

### ETT

In [4]:
from data_provider.data_loader import Dataset_ETT_minute
from data_provider.data_factory import data_provider

ettds, ettdl = data_provider(args = ett_args, flag = 'train')

train 34129


In [5]:
ettds.data_x.shape

(34560, 7)

In [6]:
ettds.__len__(), ettdl.__len__(), ettdl.batch_size

(34129, 1067, 32)

In [7]:
ettds.__getitem__(0)[2].shape

(96, 4)

In [8]:
x,y, x_mark, y_mark = next(iter(ettdl))

In [9]:
ettds.data_stamp

array([[-0.5       ,  0.16666667, -0.5       , -0.00136986],
       [-0.5       ,  0.16666667, -0.5       , -0.00136986],
       [-0.5       ,  0.16666667, -0.5       , -0.00136986],
       ...,
       [ 0.5       ,  0.5       ,  0.3       , -0.02054795],
       [ 0.5       ,  0.5       ,  0.3       , -0.02054795],
       [ 0.5       ,  0.5       ,  0.3       , -0.02054795]])

In [ ]:
x_mark[0].shape

torch.Size([96, 4])

### USC

In [44]:
uscds, uscdl = data_provider(args= usc_args, flag = 'train')
np.isnan(uscds.data_x).any(), np.isnan(uscds.data_y).any(), uscds.data_x.shape , uscds.data_x[:,:,ind_coords:].min(), uscds.data_x[:,:,ind_coords:].max()

train 23019


(False, False, (23019, 64, 32), -22.111456, 20.0733)

In [45]:
x,y, x_mark, y_mark = next(iter(uscdl))

In [46]:
x_mark[0].shape

torch.Size([64, 1])

In [47]:
uscds.data_x.shape, len(uscds.data_x),  uscdl.__len__(), uscdl.batch_size

((23019, 64, 32), 23019, 360, 64)

In [14]:
x_coords = [False]*3 + [i %4 == 3 for i in range(3, 63)]
y_coords = [False]*3 + [i %4 == 0 for i in range(3, 63)]
vx_coords = [False]*3 + [i %4 == 1 for i in range(3, 63)]
vy_coords = [False]*3 + [i %4 == 2 for i in range(3, 63)]
stats = []
for i, f in enumerate(os.listdir(args.root_path)):
    print(f" Processing file {i+1} out of {len(os.listdir(args.root_path))}", end = '\r')
    tmp = np.load(os.path.join(args.root_path, f))
    x_min, x_max, y_min, y_max = tmp[:,:,x_coords].min(), tmp[:,:,x_coords].max(), tmp[:,:,y_coords].min(), tmp[:,:,y_coords].max()
    vx_min, vx_max, vy_min, vy_max = tmp[:,:,vx_coords].min(), tmp[:,:,vx_coords].max(), tmp[:,:,vy_coords].min(), tmp[:,:,vy_coords].max()
    g_min, g_max = tmp[:,:,1:].min(), tmp[:,:,1:].max()
    stats.append({'f':f, 'shape':tmp.shape, 'min':g_min, 'max':g_max, 
                  'x_min':x_min, 'x_max':x_max, 'y_min':y_min, 'y_max':y_max, 
                  'vx_min':vx_min, 'vx_max':vx_max, 'vy_min':vy_min, 'vy_max':vy_max})
pd.DataFrame(stats)

,f,shape,min,max,x_min,x_max,y_min,y_max,vx_min,vx_max,vy_min,vy_max
0,PVTO_seq48_tar48_X.npy,"(46530, 48, 63)",-32.532787,129.939774,-29.809175,129.939774,-19.990734,89.974266,-2.993788,2.971190,-2.945130,2.968086
1,PVTO_seq32_tar32_y.npy,"(56173, 32, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.967653,89.974266,-2.984167,2.971190,-2.945130,2.994125
2,PVTO_seq24_tar24_X.npy,"(63451, 24, 63)",-55.459255,129.939774,-29.616024,129.939774,-19.952444,89.974266,-55.459255,24.223558,-27.167757,43.293716
3,PVTO_seq24_tar24_y.npy,"(63451, 24, 63)",-57.081276,129.964691,-29.653769,129.964691,-19.995951,89.974266,-55.459255,58.255981,-27.167757,43.293716
4,PVTO_seq32_tar32_X.npy,"(56173, 32, 63)",-29.809175,129.939774,-29.809175,129.939774,-19.990734,89.974266,-2.993788,2.971190,-2.945130,2.968086
5,PVTO_seq16_tar16_X.npy,"(67701, 16, 63)",-29.809175,129.952011,-29.809175,129.952011,-19.990734,89.974266,-2.993788,2.971190,-2.945130,2.994125
6,PVTO_seq64_tar64_y.npy,"(38366, 64, 63)",-57.081276,129.880692,-29.416340,129.880692,-18.236652,89.974266,-2.984167,2.947705,-2.945130,2.968086
7,PVTO_seq48_tar48_y.npy,"(46530, 48, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.967653,89.974266,-2.984167,2.971190,-2.945130,2.994125
8,PVTO_seq16_tar16_y.npy,"(67701, 16, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.988234,89.974266,-2.984167,2.971190,-2.945130,2.994125
9,PVTO_seq64_tar64_X.npy,"(38366, 64, 63)",-35.836102,129.939774,-29.809175,129.939774,-18.236652,89.974266,-2.993788,2.971190,-2.945130,2.968086


In [16]:
from models.LSTransformer import Model
from utils.model_wrapper import ModelWrapper
model = ModelWrapper(Model(configs=args), args)
model.summary()

Shape step 1 : torch.Size([1, 48, 32]) - input
Shape step 2 : torch.Size([1, 48, 256]) - post LSTM
Shape step 3 : torch.Size([1, 48, 512]) - post projection
Shape step 4 : torch.Size([1, 48, 512]) - post Transformer
Shape step 5 : torch.Size([1, 48, 256]) - post Linear
Shape step 6 : torch.Size([1, 48, 192]) - post output layer
Shape step 7 : torch.Size([1, 192, 32]) - post permute & truncate
Model quick summary
Layer name                    |          Nb units|      Nb tr params|  Nb non tr params
LSTM_0                        |               256|           296 960|                 0
Linear_0                      |               256|             8 448|                 0
LayerNorm_0                   |                 -|               512|                 0
Linear_1                      |               512|           131 584|                 0
TransformerEncoderLayer_0     |               512|         3 152 384|                 0
TransformerEncoderLayer_1     |               512|      